In [ ]:
# DataSpark : Illuminating Insights for Global Electronics

In [57]:
import pandas as pd

# Load dataset
customers = pd.read_csv('D:\\Guvi\\projects\\Data Spark\\Customers.csv', encoding='ISO-8859-1')

# Handling Missing Values
customers['City'].fillna('Unknown', inplace=True)
customers['State'].fillna('Unknown', inplace=True)
customers['State Code'].fillna('Unknown', inplace=True)  
customers['Zip Code'].fillna(0, inplace=True)
customers['Country'].fillna('Unknown', inplace=True)
customers['Continent'].fillna('Unknown', inplace=True)

# Correcting Data Types
customers['Birthday'] = pd.to_datetime(customers['Birthday'], errors='coerce')

# Removing Duplicates (excluding primary key and unique key)
customers.drop_duplicates(subset=[col for col in customers.columns if col not in ['CustomerKey']], inplace=True)

# Renaming Columns for Consistency
customers.rename(columns={'CustomerKey': 'Customer_ID', 'Zip Code': 'Zip_Code', 'State Code': 'State_Code'}, inplace=True)

# Handling Categorical Data
customers['Gender'] = customers['Gender'].astype('category')

# Calculate Age
customers['Age'] = (pd.Timestamp.now() - customers['Birthday']).dt.total_seconds() / (60*60*24*365.25)
customers['Age'] = customers['Age'].astype(int)

# Save cleaned dataset
customers.to_csv('D:\\Guvi\\projects\\Data Spark\\cleaned_customers.csv', index=False)

print("Customers data cleaning completed.")



Customers data cleaning completed.


In [40]:
import pandas as pd
import numpy as np

# Load dataset
sales = pd.read_csv('D:\\Guvi\\projects\\Data Spark\\Sales.csv', encoding='ISO-8859-1')

# Define primary and unique key columns
primary_keys_sales = ['Order Number', 'Line Item', 'CustomerKey', 'StoreKey', 'ProductKey']

# Handling Missing Values
sales.dropna(subset=['Order Date'], inplace=True)

# Correcting Data Types
sales['Order Date'] = pd.to_datetime(sales['Order Date'], errors='coerce')

# Handling Outliers (e.g., capping extreme values in Quantity)
sales['Quantity'] = np.where(sales['Quantity'] > sales['Quantity'].quantile(0.99), sales['Quantity'].quantile(0.99), sales['Quantity'])

# Removing Duplicates
sales.drop_duplicates(inplace=True)

# High Percentage of Missing Values
threshold = 0.5
columns_to_drop = [col for col in sales.columns if sales[col].isnull().sum() / len(sales) > threshold and col not in primary_keys_sales]

# Drop irrelevant, low variance, redundant, high correlation, and privacy-sensitive columns as per conditions
sales.drop(columns=set(columns_to_drop), inplace=True)

# Save cleaned dataset
sales.to_csv('D:\\Guvi\\projects\\Data Spark\\cleaned_sales.csv', index=False)

print("Sales data cleaning completed.")


Sales data cleaning completed.


In [45]:
import pandas as pd

# Load dataset
stores = pd.read_csv('D:\\Guvi\\projects\\Data Spark\\Stores.csv', encoding='ISO-8859-1')

# Display the first few rows to understand the data
print(stores.head())

# Handling Missing Values

stores['Country'].fillna('Unknown', inplace=True)
stores['State'].fillna('Unknown', inplace=True)
stores['Square Meters'].fillna(0, inplace=True)

# Correcting Data Types
stores['Open Date'] = pd.to_datetime(stores['Open Date'], errors='coerce')

# Removing Duplicates (excluding primary key)
stores.drop_duplicates(subset=[col for col in stores.columns if col not in ['StoreKey']], inplace=True)

# Handle Inconsistent Data (if any)

# Save cleaned dataset
stores.to_csv('D:\\Guvi\\projects\\Data Spark\\cleaned_stores.csv', index=False)

print("Stores data cleaning completed.")


   StoreKey    Country                         State  Square Meters  Open Date
0         1  Australia  Australian Capital Territory          595.0   1/1/2008
1         2  Australia            Northern Territory          665.0  1/12/2008
2         3  Australia               South Australia         2000.0   1/7/2012
3         4  Australia                      Tasmania         2000.0   1/1/2010
4         5  Australia                      Victoria         2000.0  12/9/2015
Stores data cleaning completed.


In [49]:
import pandas as pd

# Load dataset
products = pd.read_csv('D:\\Guvi\\projects\\Data Spark\\Products.csv', encoding='ISO-8859-1')

# Handling Missing Values
products['Color'].fillna('Unknown', inplace=True)

# Removing Duplicates
products.drop_duplicates(inplace=True)

# Remove currency symbols before converting to numeric
for column in ['Unit Cost USD', 'Unit Price USD']:
    products[column] = products[column].str.replace('$', '').str.replace(',', '')

# Correcting Data Types (e.g., converting cost and price to numeric)
for column in ['Unit Cost USD', 'Unit Price USD']:
    try:
        products[column] = pd.to_numeric(products[column], errors='coerce')
        print(f"Converted {column} to numeric successfully.")
    except Exception as e:
        print(f"Error converting {column} to numeric: {e}")

# Additional step to handle missing values in cost and price columns
for column in ['Unit Cost USD', 'Unit Price USD']:
    if products[column].isnull().any():
        products[column].fillna(products[column].median(), inplace=True)
        print(f"Filled missing values in {column} with the median.")

# Save cleaned dataset
products.to_csv('D:\\Guvi\\projects\\Data Spark\\cleaned_products.csv', index=False)

print("Products data cleaning completed.")


Converted Unit Cost USD to numeric successfully.
Converted Unit Price USD to numeric successfully.
Products data cleaning completed.


In [50]:
import pandas as pd

# Load dataset
exchange_rates = pd.read_csv('D:\\Guvi\\projects\\Data Spark\\Exchange_Rates.csv', encoding='ISO-8859-1')

# Correcting Data Types
exchange_rates['Date'] = pd.to_datetime(exchange_rates['Date'], errors='coerce')

# Removing Duplicates
exchange_rates.drop_duplicates(inplace=True)

# Save cleaned dataset
exchange_rates.to_csv('D:\\Guvi\\projects\\Data Spark\\cleaned_exchange_rates.csv', index=False)

print("Exchange rates data cleaning completed.")


Exchange rates data cleaning completed.


In [58]:
import pandas as pd

# Load cleaned datasets
customers = pd.read_csv('D:\\Guvi\\projects\\Data Spark\\cleaned_customers.csv', encoding='ISO-8859-1')
sales = pd.read_csv('D:\\Guvi\\projects\\Data Spark\\cleaned_sales.csv', encoding='ISO-8859-1')
stores = pd.read_csv('D:\\Guvi\\projects\\Data Spark\\cleaned_stores.csv', encoding='ISO-8859-1')
products = pd.read_csv('D:\\Guvi\\projects\\Data Spark\\cleaned_products.csv', encoding='ISO-8859-1')
exchange_rates = pd.read_csv('D:\\Guvi\\projects\\Data Spark\\cleaned_exchange_rates.csv', encoding='ISO-8859-1')

# Merge sales with customers
sales_customers = pd.merge(sales, customers, how='left', left_on='CustomerKey', right_on='Customer_ID')

# Merge sales_customers with stores
sales_customers_stores = pd.merge(sales_customers, stores, how='left', left_on='StoreKey', right_on='StoreKey')

# Merge sales_customers_stores with products
sales_customers_stores_products = pd.merge(sales_customers_stores, products, how='left', left_on='ProductKey', right_on='ProductKey')

# Optionally, merge with exchange rates if needed (e.g., if there are columns that need exchange rates)
# Assuming you might want to join on a common date column, which isn't directly mentioned in the given datasets
# sales_customers_stores_products = pd.merge(sales_customers_stores_products, exchange_rates, how='left', left_on='Date', right_on='Date')

# Save the merged dataset
sales_customers_stores_products.to_csv('D:\\Guvi\\projects\\Data Spark\\merged_data.csv', index=False)

print("Data merging completed.")


Data merging completed.


In [63]:
import sqlite3
import pandas as pd

# Paths to the cleaned datasets
customers_path = 'D:\\Guvi\\projects\\Data Spark\\cleaned_customers.csv'
sales_path = 'D:\\Guvi\\projects\\Data Spark\\cleaned_sales.csv'
stores_path = 'D:\\Guvi\\projects\\Data Spark\\cleaned_stores.csv'
products_path = 'D:\\Guvi\\projects\\Data Spark\\cleaned_products.csv'
exchange_rates_path = 'D:\\Guvi\\projects\\Data Spark\\cleaned_exchange_rates.csv'
merged_data_path = 'D:\\Guvi\\projects\\Data Spark\\merged_data.csv'

# Path to the SQLite database
db_path = 'D:\\Guvi\\projects\\Data Spark\\Data_Spark.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Load each dataset
customers = pd.read_csv(customers_path)
sales = pd.read_csv(sales_path)
stores = pd.read_csv(stores_path)
products = pd.read_csv(products_path)
exchange_rates = pd.read_csv(exchange_rates_path)
merged_data = pd.read_csv(merged_data_path)

# Insert each DataFrame into the SQLite database
customers.to_sql('Customers', conn, if_exists='replace', index=False)
sales.to_sql('Sales', conn, if_exists='replace', index=False)
stores.to_sql('Stores', conn, if_exists='replace', index=False)
products.to_sql('Products', conn, if_exists='replace', index=False)
exchange_rates.to_sql('ExchangeRates', conn, if_exists='replace', index=False)
merged_data.to_sql('MergedData', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print("All datasets successfully stored in the SQLite database.")


All datasets successfully stored in the SQLite database.
